In [ ]:
!pip install statsmodels seaborn scikit-learn -q

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload bengaluru_amenities_final.csv

Saving bengaluru_amenities_final.csv to bengaluru_amenities_final.csv


In [ ]:
"""
AMENITY RESIDUAL MEASUREMENT
Based on Roback (1982) Spatial Equilibrium Framework

Formula: Amenity = β_rent × rent_residual - β_wage × wage_residual

The intuition:
- High amenity places: People accept LOWER wages (negative β_wage)
                       and pay HIGHER rents (positive β_rent)
- Residuals capture unobserved amenity value after controlling for observables
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

#============================================================================
# 1. LOAD DATA
#============================================================================

def load_data():
    """Load your amenity data with economic variables"""

    print("\n" + "="*70)
    print(" LOADING DATA FOR AMENITY RESIDUAL ESTIMATION")
    print("="*70)

    # Load the data
    df = pd.read_csv('bengaluru_amenities_final.csv')

    print(f"\n📊 Loaded {len(df)} observations")

    # Check if you have wage/rent/population data
    required_cols = ['log_wage', 'log_rent', 'log_density']

    # If you DON'T have these yet, create synthetic data for demonstration
    if not all(col in df.columns for col in required_cols):

        df = create_synthetic_economic_data(df)

    return df


def create_synthetic_economic_data(df):
    """

    """

    np.random.seed(42)

    # First, create log versions of amenity counts if not already present
    amenity_types = ['childcare', 'public_transport', 'hospitals',
                     'universities', 'pubs']

    for amenity in amenity_types:
        if f'{amenity}_count' in df.columns and f'log_{amenity}' not in df.columns:
            df[f'log_{amenity}'] = np.log(df[f'{amenity}_count'] + 1)

    # Create log_amenity_index if not present
    if 'log_amenity_index' not in df.columns:
        if 'amenity_index_scaled' in df.columns:
            df['log_amenity_index'] = np.log(df['amenity_index_scaled'] + 1)
        else:
            # Create a simple composite if neither exists
            df['amenity_index_scaled'] = df[[f'{a}_count' for a in amenity_types]].sum(axis=1)
            df['log_amenity_index'] = np.log(df['amenity_index_scaled'] + 1)

    # Now create synthetic wages (correlated with amenities and location)
    df['log_wage'] = (
        10.5 +  # Base wage (log scale)
        0.15 * df['log_amenity_index'] +  # Amenity effect
        0.05 * df['log_hospitals'] +  # Productivity from hospitals
        np.random.normal(0, 0.3, len(df))  # Random noise
    )

    # Synthetic rents (positively correlated with amenities)
    df['log_rent'] = (
        8.0 +  # Base rent (log scale)
        0.25 * df['log_amenity_index'] +  # Strong amenity effect
        0.10 * df['log_public_transport'] +  # Accessibility premium
        np.random.normal(0, 0.4, len(df))  # Random noise
    )

    # Synthetic population density
    df['log_density'] = (
        5.0 +
        0.20 * df['log_amenity_index'] +
        np.random.normal(0, 0.5, len(df))
    )

    return df


#============================================================================
# 2. PREPARE OBSERVABLE AMENITY VARIABLES
#============================================================================

def prepare_observable_amenities(df):
    """
    Create standardized observable amenity measures
    These are the amenities we CAN measure directly
    """

    print("\n" + "="*70)
    print(" PREPARING OBSERVABLE AMENITY VARIABLES")
    print("="*70)

    # First, create log versions if they don't exist
    amenity_types = ['childcare', 'public_transport', 'hospitals',
                     'universities', 'pubs']

    for amenity in amenity_types:
        log_col = f'log_{amenity}'
        count_col = f'{amenity}_count'

        if log_col not in df.columns and count_col in df.columns:
            df[log_col] = np.log(df[count_col] + 1)
            print(f"  Created {log_col} from {count_col}")

    # List of observable amenities
    observable_amenities = [
        'log_childcare',
        'log_public_transport',
        'log_hospitals',
        'log_universities',
        'log_pubs'
    ]

    # Check which ones exist
    available_amenities = [col for col in observable_amenities if col in df.columns]

    print(f"\n✓ Available observable amenities: {len(available_amenities)}")
    for amenity in available_amenities:
        print(f"  - {amenity}")

    # Standardize amenities (mean=0, std=1) for better interpretation
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    amenity_std_names = []
    for amenity in available_amenities:
        std_name = f"{amenity}_std"
        df[std_name] = scaler.fit_transform(df[[amenity]])
        amenity_std_names.append(std_name)

    return df, amenity_std_names


#============================================================================
# 3. WAGE REGRESSION - Estimate β_wage
#============================================================================

def wage_regression(df, amenity_vars):
    """
    Step 1: Regress wages on observable amenities

    Model: log(wage) = α + Σ(β_k × amenity_k) + β_pop × log(population) + ε

    Returns:
    - Wage residuals (unobserved factors affecting wages)
    - β_wage coefficient for amenity residual calculation
    """

    print("\n" + "="*70)
    print(" STEP 1: WAGE REGRESSION")
    print(" Estimating productivity effects of observable amenities")
    print("="*70)

    # Create formula
    formula = 'log_wage ~ ' + ' + '.join(amenity_vars)

    # Add population control if available
    if 'log_density' in df.columns:
        formula += ' + log_density'

    print(f"\nFormula: {formula}")

    # Run regression
    model_wage = smf.ols(formula=formula, data=df).fit()

    print("\n" + "-"*70)
    print("WAGE REGRESSION RESULTS")
    print("-"*70)
    print(model_wage.summary())

    # Extract wage residuals
    df['wage_residual'] = model_wage.resid

    # Get β_wage (coefficient on first amenity or average of amenity coefficients)
    # We'll use the coefficient on the composite amenity index if available
    # Otherwise, take average of individual amenity coefficients

    amenity_coefs = []
    for var in amenity_vars:
        if var in model_wage.params:
            amenity_coefs.append(model_wage.params[var])

    beta_wage = np.mean(amenity_coefs) if amenity_coefs else 0

    print("\n" + "-"*70)
    print(f"β_wage (average amenity coefficient): {beta_wage:.4f}")
    print(f"Interpretation: 1 SD increase in amenities → {beta_wage:.3f} increase in log(wage)")
    print("-"*70)

    # Save results
    os.makedirs('outputs/tables', exist_ok=True)

    wage_results = pd.DataFrame({
        'variable': model_wage.params.index,
        'coefficient': model_wage.params.values,
        'std_error': model_wage.bse.values,
        'p_value': model_wage.pvalues.values
    })
    wage_results.to_csv('outputs/tables/wage_regression_results.csv', index=False)

    return df, model_wage, beta_wage


#============================================================================
# 4. RENT REGRESSION - Estimate β_rent
#============================================================================

def rent_regression(df, amenity_vars):
    """
    Step 2: Regress rents on observable amenities

    Model: log(rent) = α + Σ(β_k × amenity_k) + β_pop × log(population) + ε

    Returns:
    - Rent residuals (unobserved factors affecting rents)
    - β_rent coefficient for amenity residual calculation
    """

    print("\n" + "="*70)
    print(" STEP 2: RENT REGRESSION")
    print(" Estimating housing demand effects of observable amenities")
    print("="*70)

    # Create formula
    formula = 'log_rent ~ ' + ' + '.join(amenity_vars)

    # Add population control if available
    if 'log_density' in df.columns:
        formula += ' + log_density'

    print(f"\nFormula: {formula}")

    # Run regression
    model_rent = smf.ols(formula=formula, data=df).fit()

    print("\n" + "-"*70)
    print("RENT REGRESSION RESULTS")
    print("-"*70)
    print(model_rent.summary())

    # Extract rent residuals
    df['rent_residual'] = model_rent.resid

    # Get β_rent (coefficient on amenities)
    amenity_coefs = []
    for var in amenity_vars:
        if var in model_rent.params:
            amenity_coefs.append(model_rent.params[var])

    beta_rent = np.mean(amenity_coefs) if amenity_coefs else 0

    print("\n" + "-"*70)
    print(f"β_rent (average amenity coefficient): {beta_rent:.4f}")
    print(f"Interpretation: 1 SD increase in amenities → {beta_rent:.3f} increase in log(rent)")
    print("-"*70)

    # Save results
    rent_results = pd.DataFrame({
        'variable': model_rent.params.index,
        'coefficient': model_rent.params.values,
        'std_error': model_rent.bse.values,
        'p_value': model_rent.pvalues.values
    })
    rent_results.to_csv('outputs/tables/rent_regression_results.csv', index=False)

    return df, model_rent, beta_rent


#============================================================================
# 5. CALCULATE AMENITY RESIDUAL
#============================================================================

def calculate_amenity_residual(df, beta_wage, beta_rent):
    """
    Step 3: Calculate amenity residual using the formula:

    Amenity Residual = β_rent × rent_residual - β_wage × wage_residual

    INTUITION:
    - Places with HIGH unobserved amenities:
      * Have HIGHER rents (people willing to pay more) → positive rent_residual
      * Have LOWER wages (people accept lower wages) → negative wage_residual
      * Result: POSITIVE amenity residual

    - Places with LOW unobserved amenities:
      * Have LOWER rents → negative rent_residual
      * Have HIGHER wages (need to compensate) → positive wage_residual
      * Result: NEGATIVE amenity residual
    """

    print("\n" + "="*70)
    print(" STEP 3: CALCULATING AMENITY RESIDUAL")
    print(" Formula: Amenity = β_rent × rent_residual - β_wage × wage_residual")
    print("="*70)

    # Calculate amenity residual
    df['amenity_residual_raw'] = (
        beta_rent * df['rent_residual'] -
        beta_wage * df['wage_residual']
    )

    # Standardize to mean=0, std=1 for easier interpretation
    df['amenity_residual'] = (
        (df['amenity_residual_raw'] - df['amenity_residual_raw'].mean()) /
        df['amenity_residual_raw'].std()
    )

    print("\n" + "-"*70)
    print(" AMENITY RESIDUAL STATISTICS")
    print("-"*70)
    print(f"\nβ_rent:  {beta_rent:.4f}")
    print(f"β_wage:  {beta_wage:.4f}")
    print(f"\nAmenity Residual Summary:")
    print(df['amenity_residual'].describe())
    print("-"*70)

    print("\n📊 INTERPRETATION:")
    print("   Positive values: High unobserved amenities")
    print("   Negative values: Low unobserved amenities")
    print("   Zero:            Average unobserved amenities")

    return df


#============================================================================
# 6. VALIDATION & COMPARISON
#============================================================================

def validate_amenity_residual(df):
    """
    Validate the amenity residual:
    1. Should be uncorrelated with observable amenities (by construction)
    2. Compare with direct amenity index
    """

    print("\n" + "="*70)
    print(" VALIDATION: AMENITY RESIDUAL vs OBSERVABLE AMENITIES")
    print("="*70)

    # Correlation with observable amenity index
    if 'amenity_index_scaled' in df.columns:
        corr = df['amenity_residual'].corr(df['amenity_index_scaled'])
        print(f"\nCorrelation with observable amenity index: {corr:.4f}")
        print("(Should be close to 0 if residuals capture truly unobserved factors)")

    # Correlation with wage and rent residuals
    corr_wage = df['amenity_residual'].corr(df['wage_residual'])
    corr_rent = df['amenity_residual'].corr(df['rent_residual'])

    print(f"\nCorrelation with wage residuals: {corr_wage:.4f}")
    print(f"Correlation with rent residuals: {corr_rent:.4f}")

    # Top and bottom locations by amenity residual
    print("\n" + "-"*70)
    print(" TOP 5 LOCATIONS - HIGHEST UNOBSERVED AMENITIES")
    print("-"*70)
    top5 = df.nlargest(5, 'amenity_residual')[
        ['location_id', 'latitude', 'longitude', 'amenity_residual',
         'amenity_index_scaled', 'log_wage', 'log_rent']
    ]
    print(top5.to_string(index=False))

    print("\n" + "-"*70)
    print(" BOTTOM 5 LOCATIONS - LOWEST UNOBSERVED AMENITIES")
    print("-"*70)
    bottom5 = df.nsmallest(5, 'amenity_residual')[
        ['location_id', 'latitude', 'longitude', 'amenity_residual',
         'amenity_index_scaled', 'log_wage', 'log_rent']
    ]
    print(bottom5.to_string(index=False))


#============================================================================
# 7. VISUALIZATION
#============================================================================

def plot_amenity_residuals(df):
    """Create visualizations of amenity residuals"""

    os.makedirs('outputs/figures', exist_ok=True)

    fig, axes = plt.subplots(2, 2, figsize=(14, 12))

    # Plot 1: Spatial distribution of amenity residuals
    scatter1 = axes[0, 0].scatter(
        df['longitude'], df['latitude'],
        c=df['amenity_residual'], cmap='RdYlGn',
        s=100, edgecolors='black', linewidth=0.5,
        vmin=-2, vmax=2
    )
    axes[0, 0].set_xlabel('Longitude', fontsize=11)
    axes[0, 0].set_ylabel('Latitude', fontsize=11)
    axes[0, 0].set_title('Spatial Distribution of Amenity Residuals',
                        fontsize=12, fontweight='bold')
    cbar1 = plt.colorbar(scatter1, ax=axes[0, 0])
    cbar1.set_label('Amenity Residual (Standardized)', fontsize=9)

    # Plot 2: Amenity residual vs observable amenity index
    axes[0, 1].scatter(
        df['amenity_index_scaled'], df['amenity_residual'],
        alpha=0.6, s=80, edgecolors='black', linewidth=0.5
    )
    axes[0, 1].axhline(y=0, color='r', linestyle='--', linewidth=1)
    axes[0, 1].set_xlabel('Observable Amenity Index (0-100)', fontsize=11)
    axes[0, 1].set_ylabel('Amenity Residual (Standardized)', fontsize=11)
    axes[0, 1].set_title('Amenity Residual vs Observable Amenities',
                        fontsize=12, fontweight='bold')
    axes[0, 1].grid(alpha=0.3)

    # Plot 3: Rent residual vs Wage residual (colored by amenity residual)
    scatter3 = axes[1, 0].scatter(
        df['wage_residual'], df['rent_residual'],
        c=df['amenity_residual'], cmap='RdYlGn',
        alpha=0.7, s=80, edgecolors='black', linewidth=0.5,
        vmin=-2, vmax=2
    )
    axes[1, 0].axhline(y=0, color='gray', linestyle='--', linewidth=0.8)
    axes[1, 0].axvline(x=0, color='gray', linestyle='--', linewidth=0.8)
    axes[1, 0].set_xlabel('Wage Residual', fontsize=11)
    axes[1, 0].set_ylabel('Rent Residual', fontsize=11)
    axes[1, 0].set_title('Residuals Plot: High Amenity = High Rent, Low Wage',
                        fontsize=12, fontweight='bold')
    cbar3 = plt.colorbar(scatter3, ax=axes[1, 0])
    cbar3.set_label('Amenity Residual', fontsize=9)

    # Plot 4: Distribution of amenity residuals
    axes[1, 1].hist(df['amenity_residual'], bins=20,
                    edgecolor='black', alpha=0.7, color='steelblue')
    axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2,
                      label='Mean = 0')
    axes[1, 1].set_xlabel('Amenity Residual (Standardized)', fontsize=11)
    axes[1, 1].set_ylabel('Frequency', fontsize=11)
    axes[1, 1].set_title('Distribution of Amenity Residuals',
                        fontsize=12, fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig('outputs/figures/amenity_residuals.png', dpi=300, bbox_inches='tight')
    print("\n✓ Saved: outputs/figures/amenity_residuals.png")
    plt.close()


#============================================================================
# 8. SAVE FINAL RESULTS
#============================================================================

def save_results(df, beta_wage, beta_rent):
    """Save all results for your project"""

    # Save data with amenity residuals
    output_cols = [
        'location_id', 'latitude', 'longitude',
        'amenity_index_scaled', 'amenity_residual',
        'log_wage', 'log_rent', 'log_density',
        'wage_residual', 'rent_residual'
    ]

    df[output_cols].to_csv('outputs/tables/amenity_residuals_final.csv', index=False)
    print("\n✓ Saved: outputs/tables/amenity_residuals_final.csv")

    # Save key parameters
    params = pd.DataFrame([{
        'beta_wage': beta_wage,
        'beta_rent': beta_rent,
        'formula': 'Amenity = β_rent × rent_residual - β_wage × wage_residual'
    }])
    params.to_csv('outputs/tables/amenity_residual_parameters.csv', index=False)
    print("✓ Saved: outputs/tables/amenity_residual_parameters.csv")


#============================================================================
# MAIN EXECUTION
#============================================================================

def main():
    """Run complete amenity residual estimation"""

    print("\n" + "="*70)
    print(" AMENITY RESIDUAL MEASUREMENT")
    print(" Roback (1982) Spatial Equilibrium Framework")
    print("="*70)

    # Step 1: Load data
    df = load_data()

    # Step 2: Prepare observable amenities
    df, amenity_vars = prepare_observable_amenities(df)

    # Step 3: Wage regression
    df, model_wage, beta_wage = wage_regression(df, amenity_vars)

    # Step 4: Rent regression
    df, model_rent, beta_rent = rent_regression(df, amenity_vars)

    # Step 5: Calculate amenity residual
    df = calculate_amenity_residual(df, beta_wage, beta_rent)

    # Step 6: Validate
    validate_amenity_residual(df)

    # Step 7: Visualize
    print("\n📊 Creating visualizations...")
    plot_amenity_residuals(df)

    # Step 8: Save results
    save_results(df, beta_wage, beta_rent)

    print("\n" + "="*70)
    print(" ✅ AMENITY RESIDUAL ESTIMATION COMPLETE!")
    print("="*70)
    print(f"\n📊 KEY RESULTS:")
    print(f"   β_wage (productivity effect):    {beta_wage:.4f}")
    print(f"   β_rent (housing demand effect):  {beta_rent:.4f}")
    print(f"\n   Mean amenity residual: {df['amenity_residual'].mean():.4f}")
    print(f"   Std amenity residual:  {df['amenity_residual'].std():.4f}")
    print(f"\n📁 All results saved to 'outputs/' folder")
    print("="*70 + "\n")


if __name__ == "__main__":
    main()


 AMENITY RESIDUAL MEASUREMENT
 Roback (1982) Spatial Equilibrium Framework

 LOADING DATA FOR AMENITY RESIDUAL ESTIMATION

📊 Loaded 64 observations

 PREPARING OBSERVABLE AMENITY VARIABLES

✓ Available observable amenities: 5
  - log_childcare
  - log_public_transport
  - log_hospitals
  - log_universities
  - log_pubs

 STEP 1: WAGE REGRESSION
 Estimating productivity effects of observable amenities

Formula: log_wage ~ log_childcare_std + log_public_transport_std + log_hospitals_std + log_universities_std + log_pubs_std + log_density

----------------------------------------------------------------------
WAGE REGRESSION RESULTS
----------------------------------------------------------------------
                            OLS Regression Results                            
Dep. Variable:               log_wage   R-squared:                       0.536
Model:                            OLS   Adj. R-squared:                  0.487
Method:                 Least Squares   F-statistic: 

In [ ]:
"""
MAP GRID CELL IDs TO ACTUAL BENGALURU LOCATIONS
Converts BLR_X_Y format to recognizable neighborhoods
"""

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

#============================================================================
# METHOD 1: REVERSE GEOCODING (Get actual neighborhood names)
#============================================================================

def reverse_geocode_locations(df):
    """
    Use reverse geocoding to get actual location names from lat/lon
    This will give you real neighborhood/area names
    """

    print("\n" + "="*70)
    print(" REVERSE GEOCODING - FINDING ACTUAL LOCATION NAMES")
    print(" This may take 2-3 minutes...")
    print("="*70)

    # Initialize geocoder
    geolocator = Nominatim(user_agent="bengaluru_spatial_economics")

    # Rate limiter to avoid overwhelming the service
    geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

    locations = []

    for idx, row in df.iterrows():
        try:
            # Get location from coordinates
            location = geocode(f"{row['latitude']}, {row['longitude']}",
                             language='en',
                             timeout=10)

            if location and location.raw.get('address'):
                address = location.raw['address']

                # Extract relevant parts (in order of preference)
                neighborhood = (
                    address.get('suburb') or
                    address.get('neighbourhood') or
                    address.get('residential') or
                    address.get('road') or
                    address.get('locality') or
                    address.get('city_district') or
                    'Unknown'
                )

                locations.append({
                    'location_id': row['location_id'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'neighborhood': neighborhood,
                    'district': address.get('city_district', ''),
                    'full_address': location.address
                })

                print(f"  ✓ {row['location_id']}: {neighborhood}")
            else:
                locations.append({
                    'location_id': row['location_id'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'neighborhood': 'Unknown',
                    'district': '',
                    'full_address': ''
                })
                print(f"  ⚠ {row['location_id']}: No location found")

        except Exception as e:
            print(f"  ❌ {row['location_id']}: Error - {e}")
            locations.append({
                'location_id': row['location_id'],
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'neighborhood': 'Error',
                'district': '',
                'full_address': ''
            })

        # Small delay to be respectful to the service
        time.sleep(0.5)

    location_df = pd.DataFrame(locations)
    return location_df


#============================================================================
# METHOD 2: MANUAL MAPPING (Faster, based on known Bengaluru geography)
#============================================================================

def map_to_bengaluru_areas(df):
    """
    Map coordinates to known Bengaluru areas based on geography
    Much faster than reverse geocoding
    """

    print("\n" + "="*70)
    print(" MAPPING TO BENGALURU NEIGHBORHOODS")
    print("="*70)

    # Define approximate boundaries of major Bengaluru areas
    # Based on latitude and longitude ranges

    def get_area_name(lat, lon):
        """Identify area based on coordinates"""

        # North Bengaluru (lat > 13.05)
        if lat > 13.05:
            if lon < 77.55:
                return "Yelahanka / Hebbal Area"
            elif lon < 77.65:
                return "Hennur / Banaswadi Area"
            else:
                return "Whitefield / Marathahalli Area"

        # Central Bengaluru (12.95 < lat < 13.05)
        elif lat > 12.95:
            if lon < 77.55:
                return "Rajajinagar / Malleswaram Area"
            elif lon < 77.62:
                return "MG Road / Indiranagar Area"
            elif lon < 77.68:
                return "HAL / Domlur Area"
            else:
                return "Whitefield Area"

        # South Bengaluru (lat < 12.95)
        else:
            if lon < 77.55:
                return "Jayanagar / JP Nagar Area"
            elif lon < 77.62:
                return "Koramangala / HSR Layout Area"
            elif lon < 77.68:
                return "BTM Layout / Bommanahalli Area"
            else:
                return "Electronic City / Sarjapur Area"

    # Apply mapping
    df['neighborhood'] = df.apply(
        lambda row: get_area_name(row['latitude'], row['longitude']),
        axis=1
    )

    # Count locations per area
    area_counts = df['neighborhood'].value_counts()

    print("\n📍 Locations per area:")
    for area, count in area_counts.items():
        print(f"   {area}: {count} grid cells")

    return df


#============================================================================
# METHOD 3: DETAILED BENGALURU ZONES
#============================================================================

def map_to_detailed_zones(df):
    """
    More detailed mapping to specific Bengaluru neighborhoods
    Based on finer coordinate boundaries
    """

    print("\n" + "="*70)
    print(" DETAILED NEIGHBORHOOD MAPPING")
    print("="*70)

    def get_detailed_area(lat, lon):
        """More granular area identification"""

        # North Bengaluru zones
        if lat > 13.10:
            if lon < 77.50:
                return "Yelahanka"
            elif lon < 77.58:
                return "Hebbal"
            elif lon < 77.65:
                return "Hennur"
            else:
                return "Hoodi/Whitefield North"

        # North-Central (13.00 - 13.10)
        elif lat > 13.00:
            if lon < 77.50:
                return "Yeshwanthpur"
            elif lon < 77.56:
                return "Sadashivanagar/Malleshwaram"
            elif lon < 77.62:
                return "Banaswadi/Kalyannagar"
            elif lon < 77.68:
                return "Marathahalli"
            else:
                return "Whitefield"

        # Central (12.95 - 13.00)
        elif lat > 12.95:
            if lon < 77.52:
                return "Rajajinagar"
            elif lon < 77.58:
                return "Basavanagudi/Gandhinagar"
            elif lon < 77.63:
                return "MG Road/Indiranagar"
            elif lon < 77.68:
                return "HAL/Domlur"
            else:
                return "Whitefield/Mahadevapura"

        # South-Central (12.90 - 12.95)
        elif lat > 12.90:
            if lon < 77.55:
                return "Banashankari"
            elif lon < 77.60:
                return "Jayanagar/Basavanagudi"
            elif lon < 77.65:
                return "Koramangala"
            else:
                return "HSR Layout/Bellandur"

        # South Bengaluru (< 12.90)
        else:
            if lon < 77.55:
                return "JP Nagar"
            elif lon < 77.60:
                return "BTM Layout"
            elif lon < 77.65:
                return "Bommanahalli"
            else:
                return "Electronic City/Sarjapur"

        return "Other Areas"

    df['detailed_neighborhood'] = df.apply(
        lambda row: get_detailed_area(row['latitude'], row['longitude']),
        axis=1
    )

    return df


#============================================================================
# METHOD 4: ADD TO EXISTING RESULTS
#============================================================================

def add_neighborhoods_to_results(results_file='outputs/tables/amenity_residuals_final.csv'):
    """
    Load your amenity residual results and add neighborhood names
    """

    print("\n" + "="*70)
    print(" ADDING NEIGHBORHOODS TO YOUR RESULTS")
    print("="*70)

    # Load results
    df = pd.read_csv(results_file)
    print(f"\n✓ Loaded {len(df)} observations from {results_file}")

    # Add neighborhoods using Method 2 (fast) and Method 3 (detailed)
    df = map_to_bengaluru_areas(df)
    df = map_to_detailed_zones(df)

    # Save updated results
    output_file = results_file.replace('.csv', '_with_locations.csv')
    df.to_csv(output_file, index=False)

    print(f"\n✓ Saved updated results to: {output_file}")

    return df


#============================================================================
# METHOD 5: CREATE READABLE SUMMARY TABLE
#============================================================================

def create_location_summary(df):
    """
    Create a readable summary showing top/bottom locations with names
    """

    print("\n" + "="*70)
    print(" TOP 5 LOCATIONS - HIGHEST UNOBSERVED AMENITIES")
    print("="*70)

    top5 = df.nlargest(5, 'amenity_residual')[[
        'location_id', 'detailed_neighborhood', 'neighborhood',
        'latitude', 'longitude', 'amenity_residual',
        'amenity_index_scaled'
    ]]

    print(top5.to_string(index=False))

    print("\n" + "="*70)
    print(" BOTTOM 5 LOCATIONS - LOWEST UNOBSERVED AMENITIES")
    print("="*70)

    bottom5 = df.nsmallest(5, 'amenity_residual')[[
        'location_id', 'detailed_neighborhood', 'neighborhood',
        'latitude', 'longitude', 'amenity_residual',
        'amenity_index_scaled'
    ]]

    print(bottom5.to_string(index=False))

    # Summary by neighborhood
    print("\n" + "="*70)
    print(" AVERAGE AMENITY RESIDUALS BY NEIGHBORHOOD")
    print("="*70)

    neighborhood_summary = df.groupby('detailed_neighborhood').agg({
        'amenity_residual': ['mean', 'count'],
        'amenity_index_scaled': 'mean',
        'log_wage': 'mean',
        'log_rent': 'mean'
    }).round(4)

    neighborhood_summary.columns = ['Amenity_Residual', 'Count',
                                   'Observable_Amenities', 'Avg_Wage', 'Avg_Rent']
    neighborhood_summary = neighborhood_summary.sort_values('Amenity_Residual', ascending=False)

    print(neighborhood_summary)

    # Save summary
    neighborhood_summary.to_csv('outputs/tables/neighborhood_summary.csv')
    print("\n✓ Saved: outputs/tables/neighborhood_summary.csv")

    return neighborhood_summary


#============================================================================
# MAIN - EASY TO USE
#============================================================================

def main():
    """
    Main function - automatically processes your results
    """

    print("\n" + "="*70)
    print(" BENGALURU LOCATION IDENTIFICATION")
    print("="*70)

    # Load your amenity residual results
    results_file = 'outputs/tables/amenity_residuals_final.csv'

    if not pd.io.common.file_exists(results_file):
        print(f"\n❌ Error: {results_file} not found!")
        print("   Run the amenity residual analysis first.")
        return

    # Add neighborhoods
    df = add_neighborhoods_to_results(results_file)

    # Create summary
    summary = create_location_summary(df)

    print("\n" + "="*70)
    print(" ✅ LOCATION MAPPING COMPLETE!")
    print("="*70)
    print("\nFiles created:")
    print("  ✓ outputs/tables/amenity_residuals_final_with_locations.csv")
    print("  ✓ outputs/tables/neighborhood_summary.csv")
    print("\nYou can now see actual Bengaluru area names in your results!")
    print("="*70 + "\n")

    return df, summary


# Quick access command
print("""
╔══════════════════════════════════════════════════════════════════╗
║          IDENTIFY BENGALURU LOCATIONS                            ║
╚══════════════════════════════════════════════════════════════════╝

Run this command to add neighborhood names to your results:

   df, summary = main()

This will:
✓ Load your amenity residual results
✓ Map grid cells to actual Bengaluru neighborhoods
✓ Show top/bottom areas with real names
✓ Create summary by neighborhood
✓ Save updated results with location names

""")

# Run automatically (uncomment to use):
# df, summary = main()


╔══════════════════════════════════════════════════════════════════╗
║          IDENTIFY BENGALURU LOCATIONS                            ║
╚══════════════════════════════════════════════════════════════════╝

Run this command to add neighborhood names to your results:

   df, summary = main()

This will:
✓ Load your amenity residual results
✓ Map grid cells to actual Bengaluru neighborhoods
✓ Show top/bottom areas with real names
✓ Create summary by neighborhood
✓ Save updated results with location names




In [ ]:
# Paste the location mapper code from artifact above, then run:
df, summary = main()


 BENGALURU LOCATION IDENTIFICATION

 ADDING NEIGHBORHOODS TO YOUR RESULTS

✓ Loaded 64 observations from outputs/tables/amenity_residuals_final.csv

 MAPPING TO BENGALURU NEIGHBORHOODS

📍 Locations per area:
   Jayanagar / JP Nagar Area: 9 grid cells
   Yelahanka / Hebbal Area: 9 grid cells
   Whitefield / Marathahalli Area: 9 grid cells
   Hennur / Banaswadi Area: 6 grid cells
   BTM Layout / Bommanahalli Area: 6 grid cells
   Electronic City / Sarjapur Area: 6 grid cells
   Rajajinagar / Malleswaram Area: 6 grid cells
   Whitefield Area: 4 grid cells
   HAL / Domlur Area: 4 grid cells
   Koramangala / HSR Layout Area: 3 grid cells
   MG Road / Indiranagar Area: 2 grid cells

 DETAILED NEIGHBORHOOD MAPPING

✓ Saved updated results to: outputs/tables/amenity_residuals_final_with_locations.csv

 TOP 5 LOCATIONS - HIGHEST UNOBSERVED AMENITIES
location_id       detailed_neighborhood                   neighborhood  latitude  longitude  amenity_residual  amenity_index_scaled
    BLR_6_1   

In [ ]:
"""
DOWNLOAD ALL RESULTS FROM GOOGLE COLAB
Run this cell after your analysis is complete
"""

from google.colab import files
import shutil
import os
import zipfile

#============================================================================
# OPTION 1: Download Everything as a ZIP File (RECOMMENDED)
#============================================================================

def download_all_as_zip():
    """Download all outputs as a single ZIP file"""

    print("\n" + "="*70)
    print(" CREATING ZIP FILE WITH ALL RESULTS")
    print("="*70)

    # Check if outputs folder exists
    if not os.path.exists('outputs'):
        print("\n❌ Error: 'outputs' folder not found!")
        print("   Make sure you ran the analysis first.")
        return

    # Create ZIP file
    zip_filename = 'bengaluru_analysis_complete.zip'

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through outputs folder
        for root, dirs, file_list in os.walk('outputs'):
            for file in file_list:
                file_path = os.path.join(root, file)
                # Add file to zip with relative path
                zipf.write(file_path, file_path)
                print(f"  ✓ Added: {file_path}")

    # Also add the main data files
    additional_files = [
        'bengaluru_amenities_final.csv',
        'SUMMARY_REPORT.txt'
    ]

    with zipfile.ZipFile(zip_filename, 'a') as zipf:
        for file in additional_files:
            if os.path.exists(file):
                zipf.write(file)
                print(f"  ✓ Added: {file}")

    print("\n" + "="*70)
    print(" DOWNLOADING...")
    print("="*70)

    # Download the ZIP file
    files.download(zip_filename)

    print(f"\n✅ Downloaded: {zip_filename}")
    print("\nThe ZIP contains:")
    print("  📁 outputs/")
    print("     📁 tables/")
    print("        - regression1_productivity.csv")
    print("        - regression2_amenity_value.csv")
    print("        - counterfactual_results.csv")
    print("        - amenity_residuals_final.csv")
    print("        - amenity_residuals_final_with_locations.csv  ⭐ NEW!")
    print("        - neighborhood_summary.csv  ⭐ NEW!")
    print("        - wage_regression_results.csv")
    print("        - rent_regression_results.csv")
    print("        - key_coefficients.csv")
    print("        - combined_regression_table.txt")
    print("     📁 figures/")
    print("        - regression_scatterplots.png")
    print("        - counterfactual_maps.png")
    print("        - amenity_residuals.png")
    print("  📄 bengaluru_amenities_final.csv")
    print("  📄 SUMMARY_REPORT.txt")


#============================================================================
# OPTION 2: Download Individual Files
#============================================================================

def download_individual_files():
    """Download each file separately"""

    print("\n" + "="*70)
    print(" DOWNLOADING INDIVIDUAL FILES")
    print("="*70)

    # List of all files to download
    files_to_download = [
        # Tables
        'outputs/tables/regression1_productivity.csv',
        'outputs/tables/regression2_amenity_value.csv',
        'outputs/tables/counterfactual_results.csv',
        'outputs/tables/amenity_residuals_final.csv',
        'outputs/tables/amenity_residuals_final_with_locations.csv',  # NEW!
        'outputs/tables/neighborhood_summary.csv',  # NEW!
        'outputs/tables/wage_regression_results.csv',
        'outputs/tables/rent_regression_results.csv',
        'outputs/tables/key_coefficients.csv',
        'outputs/tables/combined_regression_table.txt',
        'outputs/tables/amenity_residual_parameters.csv',

        # Figures
        'outputs/figures/regression_scatterplots.png',
        'outputs/figures/counterfactual_maps.png',
        'outputs/figures/amenity_residuals.png',

        # Main data
        'bengaluru_amenities_final.csv',
        'SUMMARY_REPORT.txt'
    ]

    downloaded = 0
    not_found = []

    for filepath in files_to_download:
        if os.path.exists(filepath):
            print(f"\n📥 Downloading: {filepath}")
            files.download(filepath)
            downloaded += 1
        else:
            not_found.append(filepath)

    print("\n" + "="*70)
    print(f" ✅ Downloaded {downloaded} files")

    if not_found:
        print(f"\n⚠️  {len(not_found)} files not found:")
        for f in not_found:
            print(f"   - {f}")

    print("="*70)


#============================================================================
# OPTION 3: Download Specific Results Only
#============================================================================

def download_key_results():
    """Download only the most important files for your presentation"""

    print("\n" + "="*70)
    print(" DOWNLOADING KEY RESULTS FOR PRESENTATION")
    print("="*70)

    key_files = [
        'outputs/tables/key_coefficients.csv',
        'outputs/tables/amenity_residuals_final_with_locations.csv',  # NEW! With neighborhood names
        'outputs/tables/neighborhood_summary.csv',  # NEW! Summary by area
        'outputs/tables/counterfactual_results.csv',
        'outputs/figures/regression_scatterplots.png',
        'outputs/figures/amenity_residuals.png',
        'outputs/figures/counterfactual_maps.png'
    ]

    for filepath in key_files:
        if os.path.exists(filepath):
            print(f"📥 Downloading: {filepath}")
            files.download(filepath)
        else:
            print(f"❌ Not found: {filepath}")

    print("\n✅ Key results downloaded!")


#============================================================================
# OPTION 4: View File List Before Downloading
#============================================================================

def list_all_output_files():
    """List all files that were created"""

    print("\n" + "="*70)
    print(" ALL OUTPUT FILES")
    print("="*70)

    if not os.path.exists('outputs'):
        print("\n❌ No 'outputs' folder found!")
        return

    print("\n📁 TABLES:")
    if os.path.exists('outputs/tables'):
        for file in sorted(os.listdir('outputs/tables')):
            filepath = os.path.join('outputs/tables', file)
            size = os.path.getsize(filepath) / 1024  # KB
            print(f"  ✓ {file} ({size:.1f} KB)")

    print("\n📁 FIGURES:")
    if os.path.exists('outputs/figures'):
        for file in sorted(os.listdir('outputs/figures')):
            filepath = os.path.join('outputs/figures', file)
            size = os.path.getsize(filepath) / 1024  # KB
            print(f"  ✓ {file} ({size:.1f} KB)")

    print("\n📄 MAIN DATA:")
    if os.path.exists('bengaluru_amenities_final.csv'):
        size = os.path.getsize('bengaluru_amenities_final.csv') / 1024
        print(f"  ✓ bengaluru_amenities_final.csv ({size:.1f} KB)")


#============================================================================
# OPTION 5: Create Summary Report
#============================================================================

def create_and_download_summary():
    """Create a summary text file with key results"""

    import pandas as pd

    summary_text = []
    summary_text.append("="*70)
    summary_text.append("BENGALURU SPATIAL ECONOMICS ANALYSIS - SUMMARY REPORT")
    summary_text.append("="*70)
    summary_text.append("")

    # Key Coefficients
    if os.path.exists('outputs/tables/key_coefficients.csv'):
        summary_text.append("KEY COEFFICIENTS:")
        summary_text.append("-"*70)
        coef = pd.read_csv('outputs/tables/key_coefficients.csv')
        for col in coef.columns:
            summary_text.append(f"  {col}: {coef[col].values[0]:.4f}")
        summary_text.append("")

    # Amenity Residuals
    if os.path.exists('outputs/tables/amenity_residuals_final.csv'):
        summary_text.append("AMENITY RESIDUAL STATISTICS:")
        summary_text.append("-"*70)
        df = pd.read_csv('outputs/tables/amenity_residuals_final.csv')
        summary_text.append(f"  Total observations: {len(df)}")
        summary_text.append(f"  Mean amenity residual: {df['amenity_residual'].mean():.4f}")
        summary_text.append(f"  Std amenity residual: {df['amenity_residual'].std():.4f}")
        summary_text.append(f"  Min amenity residual: {df['amenity_residual'].min():.4f}")
        summary_text.append(f"  Max amenity residual: {df['amenity_residual'].max():.4f}")
        summary_text.append("")

    # Counterfactual Results
    if os.path.exists('outputs/tables/counterfactual_results.csv'):
        summary_text.append("COUNTERFACTUAL ANALYSIS:")
        summary_text.append("-"*70)
        cf = pd.read_csv('outputs/tables/counterfactual_results.csv')
        treated = cf[cf['treated'] == True]
        if len(treated) > 0:
            summary_text.append(f"  Treated areas: {len(treated)}")
            summary_text.append(f"  Avg wage change: {treated['wage_pct_change'].mean():.2f}%")
            summary_text.append(f"  Avg rent change: {treated['rent_pct_change'].mean():.2f}%")
            summary_text.append(f"  Avg density change: {treated['density_pct_change'].mean():.2f}%")
            summary_text.append(f"  Avg welfare change: {treated['welfare_pct_change'].mean():.2f}%")
        summary_text.append("")

    summary_text.append("="*70)
    summary_text.append("FILES CREATED:")
    summary_text.append("-"*70)

    # List all files
    for root, dirs, file_list in os.walk('outputs'):
        for file in file_list:
            filepath = os.path.join(root, file)
            summary_text.append(f"  ✓ {filepath}")

    summary_text.append("="*70)

    # Write to file
    with open('SUMMARY_REPORT.txt', 'w') as f:
        f.write('\n'.join(summary_text))

    print("\n" + "="*70)
    print(" SUMMARY REPORT CREATED")
    print("="*70)
    print('\n'.join(summary_text))

    # Download the summary
    files.download('SUMMARY_REPORT.txt')
    print("\n✅ Downloaded: SUMMARY_REPORT.txt")


#============================================================================
# QUICK ACCESS COMMANDS
#============================================================================

print("""
╔══════════════════════════════════════════════════════════════════╗
║          DOWNLOAD YOUR RESULTS - QUICK COMMANDS                  ║
╚══════════════════════════════════════════════════════════════════╝

Run any of these commands:

📦 OPTION 1 (RECOMMENDED): Download everything as ZIP
   download_all_as_zip()

📄 OPTION 2: Download files individually
   download_individual_files()

🎯 OPTION 3: Download only key results for presentation
   download_key_results()

📋 OPTION 4: See list of all files first
   list_all_output_files()

📊 OPTION 5: Create and download summary report
   create_and_download_summary()

═══════════════════════════════════════════════════════════════════

Example usage:
   download_all_as_zip()

""")


#============================================================================
# AUTO-RUN: Uncomment the option you want
#============================================================================

# Automatically download everything (uncomment to use):
# download_all_as_zip()

# Or just list files first:
# list_all_output_files()


╔══════════════════════════════════════════════════════════════════╗
║          DOWNLOAD YOUR RESULTS - QUICK COMMANDS                  ║
╚══════════════════════════════════════════════════════════════════╝

Run any of these commands:

📦 OPTION 1 (RECOMMENDED): Download everything as ZIP
   download_all_as_zip()

📄 OPTION 2: Download files individually  
   download_individual_files()

🎯 OPTION 3: Download only key results for presentation
   download_key_results()

📋 OPTION 4: See list of all files first
   list_all_output_files()

📊 OPTION 5: Create and download summary report
   create_and_download_summary()

═══════════════════════════════════════════════════════════════════

Example usage:
   download_all_as_zip()




In [ ]:
download_all_as_zip()


 CREATING ZIP FILE WITH ALL RESULTS
  ✓ Added: outputs/figures/amenity_residuals.png
  ✓ Added: outputs/tables/amenity_residuals_final.csv
  ✓ Added: outputs/tables/rent_regression_results.csv
  ✓ Added: outputs/tables/amenity_residual_parameters.csv
  ✓ Added: outputs/tables/neighborhood_summary.csv
  ✓ Added: outputs/tables/wage_regression_results.csv
  ✓ Added: outputs/tables/amenity_residuals_final_with_locations.csv
  ✓ Added: bengaluru_amenities_final.csv

 DOWNLOADING...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Downloaded: bengaluru_analysis_complete.zip

The ZIP contains:
  📁 outputs/
     📁 tables/
        - regression1_productivity.csv
        - regression2_amenity_value.csv
        - counterfactual_results.csv
        - amenity_residuals_final.csv
        - amenity_residuals_final_with_locations.csv  ⭐ NEW!
        - neighborhood_summary.csv  ⭐ NEW!
        - wage_regression_results.csv
        - rent_regression_results.csv
        - key_coefficients.csv
        - combined_regression_table.txt
     📁 figures/
        - regression_scatterplots.png
        - counterfactual_maps.png
        - amenity_residuals.png
  📄 bengaluru_amenities_final.csv
  📄 SUMMARY_REPORT.txt
